In [45]:
%matplotlib inline
import matplotlib.pyplot as plt

from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo
from owslib.wfs import WebFeatureService

from pydov.search.boring import BoringSearch
from pydov.search.grondwaterfilter import GrondwaterFilterSearch
from pydov.search.grondwatermonster import GrondwaterMonsterSearch

from pydov.util.location import (
    GmlFilter,
    Within,
)

# Download data from a certain community using its geographic borders

In [16]:
gemeentegrenzen = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VRBG/wfs',
    version='1.1.0')

In [17]:
gemeentegrenzen.get_schema('VRBG:Refgem')['properties']

{'UIDN': 'decimal',
 'OIDN': 'decimal',
 'TERRID': 'decimal',
 'NISCODE': 'string',
 'NAAM': 'string',
 'DATPUBLBS': 'date',
 'NUMAC': 'string'}

In [ ]:
naam_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Lievegem')

In [ ]:
gemeente_poly = gemeentegrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

In [ ]:
bs = BoringSearch()
df = bs.search(
    location=GmlFilter(gemeente_poly, Within),
    return_fields=('pkey_boring', 'gemeente'))

In [ ]:
df.groupby('gemeente').size().reset_index(name='counts')

# Download data van bekken

In [ ]:
# Initialise WFS service
bekkens = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VHAZones/wfs',
    version='1.1.0'
)

In [ ]:
# List available types (layers)
list(bekkens.contents)

In [ ]:
# Get available fields (properties) of a layer
bekkens.get_schema('VHAZones:Bekken')['properties']

In [ ]:
# Get distinct values of a field
tree = etree.fromstring(bekkens.getfeature('VHAZones:Bekken', propertyname='BEKNAAM').read().encode("utf8"))
set((i.text for i in tree.findall('.//{%s}BEKNAAM' % tree.nsmap['VHAZones'])))

In [ ]:
# Build filter
naam_filter = PropertyIsEqualTo(propertyname='BEKNAAM', literal='Bekken Brugse polders')

In [ ]:
# Get features matching filter
bekken_poly = bekkens.getfeature(
    typename='VHAZones:Bekken',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

In [ ]:
# Get DOV data based on feature geometry
filter_search = GrondwaterFilterSearch()
df = filter_search.search(
    location=GmlFilter(bekken_poly, Within),
    return_fields=('pkey_filter', 'x', 'y')
)

In [ ]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform

In [ ]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y']))
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [ ]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_filter'][loc]).add_to(marker_cluster)
fmap

# Download data from Grondwaterlichamen

In [5]:
hhz = WebFeatureService(
    'https://www.dov.vlaanderen.be/geoserver/wfs',
    version='1.1.0')

In [27]:
hhz.get_schema("gw_beleid:gwlichamen")

{'properties': {'id': 'long',
  'grondwaterlichaam_afkorting': 'string',
  'grondwaterlichaam': 'string',
  'grondwatersysteem': 'string',
  'EU_code': 'string',
  'stroomgebiedsdistrict': 'string',
  'horizonten': 'string'},
 'geometry': '3D MultiPolygon',
 'geometry_column': 'geom'}

In [40]:
grondwaterlichaam_filter = PropertyIsEqualTo(propertyname='grondwaterlichaam_afkorting', literal='BLKS_0600_GWL_1')

In [54]:
grondwaterlichaam_poly = hhz.getfeature(
    typename='gw_beleid:gwlichamen',
    filter=etree.tostring(grondwaterlichaam_filter.toXML()).decode("utf8")).read()

In [58]:
gemeentegrenzen = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VRBG/wfs',
    version='1.1.0')

In [75]:
naam_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Hoegaarden')

In [76]:
gemeente_poly = gemeentegrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

In [77]:
from owslib.fes import And

In [78]:
ms = GrondwaterMonsterSearch()
df = ms.search(
    location=And([GmlFilter(grondwaterlichaam_poly, Within), GmlFilter(gemeente_poly, Within)]),
    return_fields=('pkey_grondwatermonster', 'x', 'y', 'gemeente', 'datum_monstername'))

In [79]:
df.groupby('gemeente').size().reset_index(name='counts')

,gemeente,counts
0,Hoegaarden,495
